# List of Exposures with Hologram at USDF


- work with Weakly_2023_01
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/19
- update : 2023/02/01



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-5.1.0/bin/python3


In [2]:
print(sys.path)

['/home/d/dagoret/notebooks/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2023_01/Spectractor/lib/python', '/home/d/dagoret/repos/repos_w_2023_01/atmospec/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-5.1.0/Linux64/obs_lsst/g81f1fedfbb+e5435568f8/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-5.1.0/Linux64/pipe_tasks/g67fd32f368+06ff137360/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-5.1.0/Linux64/shapelet/g082f9a06f4+23c005d4a7/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-5.1.0/Linux64/meas_modelfit/ge3ed7fa540+fbc60da186/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-5.1.0/Linux64/meas_extensions_shapeHSM/g9e404089df+ce4c866f0f/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-5.1.0/Linux64/scarlet_extensions/g9d18589735+87f79c168d/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-5.1.0/Linux64/proxmin/g33b4157f25/lib/python', '/opt/lsst/software

In [3]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_01/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-5.1.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2023_01/Spectractor 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

w_2023_01
   g754a7f0350+935911ccad 	current w_2023_01 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits
from astropy.time import Time

In [6]:
from IPython.display import display, HTML

In [7]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [8]:
import lsst.daf.butler as dafButler

In [9]:
#repo = "/sdf/group/rubin/repo/main"
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [10]:
collection='LATISS/raw/all'

## List of Exposures

In [11]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start','time_end' ,'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','science_program','jd','mjd'])

In [12]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    #df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,info.timespan.begin.to_string(),info.timespan.end.to_string() ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
    #                         info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle, info.science_program]
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,pd.to_datetime(info.timespan.begin.to_string()),pd.to_datetime(info.timespan.end.to_string()) ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle, info.science_program,
                             info.timespan.begin.jd,info.timespan.begin.mjd ]
    
    if count < 5:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
        
        mjd = Time(info.timespan.begin.to_string()).mjd
        jd = Time(info.timespan.begin.to_string()).jd
        print(mjd,jd)
    

----------------------------------------------------- 0 ---------------------------------------------------------
exposure:
  instrument: 'LATISS'
  id: 2022083100004
  physical_filter: 'unknown~unknown'
  obs_id: 'AT_O_20220831_000004'
  exposure_time: 0.0
  dark_time: 0.0160482
  observation_type: 'bias'
  observation_reason: 'bias'
  day_obs: 20220831
  seq_num: 4
  seq_start: 4
  seq_end: 4
  group_name: '2022083100004'
  group_id: 2022083100004
  target_name: 'UNKNOWN'
  science_program: 'unknown'
  tracking_ra: None
  tracking_dec: None
  sky_angle: None
  azimuth: None
  zenith_angle: None
  has_simulated: False
  timespan: Timespan(begin=astropy.time.Time('2022-08-31 22:58:13.633984', scale='tai', format='iso'), end=astropy.time.Time('2022-08-31 22:58:13.650000', scale='tai', format='iso'))
	 id:                   2022083100004
	 day_obs:              20220831
	 seq_num:              4
	 type-of-observation:  bias
	 target:               UNKNOWN
59822.957102245186 2459823.45710

In [13]:
info.toDict()

{'instrument': 'LATISS',
 'id': 2023020200151,
 'physical_filter': 'OG550_65mm_1~empty',
 'obs_id': 'AT_O_20230202_000151',
 'exposure_time': 0.0,
 'dark_time': 0.00671029,
 'observation_type': 'bias',
 'observation_reason': 'bias',
 'day_obs': 20230202,
 'seq_num': 151,
 'seq_start': 151,
 'seq_end': 151,
 'group_name': '2023-02-02T20:35:12.853',
 'group_id': 2552997128530000,
 'target_name': 'FlatField position',
 'science_program': 'unknown',
 'tracking_ra': None,
 'tracking_dec': None,
 'sky_angle': None,
 'azimuth': None,
 'zenith_angle': None,
 'has_simulated': False,
 'timespan': Timespan(begin=astropy.time.Time('2023-02-02 20:35:12.929007', scale='tai', format='iso'), end=astropy.time.Time('2023-02-02 20:35:12.935000', scale='tai', format='iso'))}

In [14]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34896,2023020200150,AT_O_20230202_000150,20230202,150,2023-02-02 20:35:10.524997,2023-02-02 20:35:10.534,bias,FlatField position,OG550_65mm_1~empty,None,0.0,None,None,None,unknown,2.459978e+06,59977.857761
34897,2023020200145,AT_O_20230202_000145,20230202,145,2023-02-02 20:34:58.501003,2023-02-02 20:34:58.508,bias,FlatField position,OG550_65mm_1~empty,None,0.0,None,None,None,unknown,2.459978e+06,59977.857622
34898,2023020200147,AT_O_20230202_000147,20230202,147,2023-02-02 20:35:03.303994,2023-02-02 20:35:03.311,bias,FlatField position,OG550_65mm_1~empty,None,0.0,None,None,None,unknown,2.459978e+06,59977.857677
34899,2023020200149,AT_O_20230202_000149,20230202,149,2023-02-02 20:35:08.125010,2023-02-02 20:35:08.131,bias,FlatField position,OG550_65mm_1~empty,None,0.0,None,None,None,unknown,2.459978e+06,59977.857733


In [15]:
df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

In [16]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34896,2023020200150,AT_O_20230202_000150,20230202,150,2023-02-02 20:35:10.524997,2023-02-02 20:35:10.534,bias,FlatField position,OG550_65mm_1~empty,None,0.0,None,None,None,unknown,2.459978e+06,59977.857761
34897,2023020200145,AT_O_20230202_000145,20230202,145,2023-02-02 20:34:58.501003,2023-02-02 20:34:58.508,bias,FlatField position,OG550_65mm_1~empty,None,0.0,None,None,None,unknown,2.459978e+06,59977.857622
34898,2023020200147,AT_O_20230202_000147,20230202,147,2023-02-02 20:35:03.303994,2023-02-02 20:35:03.311,bias,FlatField position,OG550_65mm_1~empty,None,0.0,None,None,None,unknown,2.459978e+06,59977.857677
34899,2023020200149,AT_O_20230202_000149,20230202,149,2023-02-02 20:35:08.125010,2023-02-02 20:35:08.131,bias,FlatField position,OG550_65mm_1~empty,None,0.0,None,None,None,unknown,2.459978e+06,59977.857733


## Selection of science

In [17]:
df_science = df_exposure[df_exposure.type == 'science']

In [18]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_1877/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [19]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
34697,2023020100798,AT_O_20230201_000798,20230201,798,2023-02-02 07:11:41.411994,2023-02-02 07:12:11.655,science,SDSS-J081508_015,SDSSr_65mm~empty,56.964661,30.0,123.976514,7.24764,48.1281,AUXTEL_DRP_IMAGING,2.459978e+06,59977.299785
34682,2023020100799,AT_O_20230201_000799,20230201,799,2023-02-02 07:12:14.038009,2023-02-02 07:12:44.265,science,SDSS-J081508_015,SDSSr_65mm~empty,57.061137,30.0,123.976433,7.247579,48.128116,AUXTEL_DRP_IMAGING,2.459978e+06,59977.300162
34698,2023020100800,AT_O_20230201_000800,20230201,800,2023-02-02 07:13:01.813006,2023-02-02 07:13:32.045,science,SDSS-J081508_037,SDSSr_65mm~empty,57.535622,30.0,123.787051,7.529642,47.969458,AUXTEL_DRP_IMAGING,2.459978e+06,59977.300715
34683,2023020100801,AT_O_20230201_000801,20230201,801,2023-02-02 07:13:34.431015,2023-02-02 07:14:04.681,science,SDSS-J081508_037,SDSSr_65mm~empty,57.635352,30.0,123.786943,7.529606,47.969508,AUXTEL_DRP_IMAGING,2.459978e+06,59977.301093
34699,2023020100802,AT_O_20230201_000802,20230201,802,2023-02-02 07:14:21.615993,2023-02-02 07:14:51.849,science,SDSS-J081508_016,SDSSr_65mm~empty,57.379314,30.0,124.071214,7.247623,47.810151,AUXTEL_DRP_IMAGING,2.459978e+06,59977.301639
34684,2023020100803,AT_O_20230201_000803,20230201,803,2023-02-02 07:14:54.232996,2023-02-02 07:15:24.462,science,SDSS-J081508_016,SDSSr_65mm~empty,57.479263,30.0,124.071246,7.247581,47.81013,AUXTEL_DRP_IMAGING,2.459978e+06,59977.302017
34700,2023020100804,AT_O_20230201_000804,20230201,804,2023-02-02 07:15:43.416998,2023-02-02 07:16:13.658,science,SDSS-J081508_023,SDSSr_65mm~empty,57.760709,30.0,123.976469,7.341593,47.618769,AUXTEL_DRP_IMAGING,2.459978e+06,59977.302586
34685,2023020100805,AT_O_20230201_000805,20230201,805,2023-02-02 07:16:16.041001,2023-02-02 07:16:46.270,science,SDSS-J081508_023,SDSSr_65mm~empty,57.857941,30.0,123.976461,7.341546,47.618819,AUXTEL_DRP_IMAGING,2.459978e+06,59977.302963
34701,2023020100806,AT_O_20230201_000806,20230201,806,2023-02-02 07:17:05.407018,2023-02-02 07:17:35.655,science,SDSS-J081508_024,SDSSr_65mm~empty,57.940958,30.0,124.071328,7.34169,47.48694,AUXTEL_DRP_IMAGING,2.459978e+06,59977.303535
34686,2023020100807,AT_O_20230201_000807,20230201,807,2023-02-02 07:17:38.040999,2023-02-02 07:18:08.267,science,SDSS-J081508_024,SDSSr_65mm~empty,58.038446,30.0,124.071239,7.341621,47.486978,AUXTEL_DRP_IMAGING,2.459978e+06,59977.303913


In [20]:
df_science.day_obs.unique()

array([20220912, 20220913, 20220914, 20220927, 20220928, 20220929,
       20221011, 20221012, 20221013, 20221025, 20221026, 20221027,
       20221108, 20221109, 20221110, 20221122, 20221123, 20221124,
       20221207, 20221208, 20221209, 20221210, 20221212, 20230117,
       20230118, 20230119, 20230131, 20230201])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [21]:
#date_sel = 20220630
#date_sel = 20220630
#date_sel = 20221208
#date_sel = 20221209

# data in january
#date_sel= 20230118
#date_sel= 20230131
date_sel = 20230201

In [22]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [23]:
len(df_science_selected)

364

# List of filters

In [24]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['SDSSr_65mm~empty', 'empty~holo4_003', 'BG40_65mm_1~holo4_003',
       'OG550_65mm_1~holo4_003'], dtype=object)

In [25]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['empty~holo4_003', 'BG40_65mm_1~holo4_003',
       'OG550_65mm_1~holo4_003'], dtype='<U22')

## Selection of filter

- But now we have a loop on filters

In [ ]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

empty~holo4_003
visitdispersers_20230201_filt_empty-holo4_003.list
BG40_65mm_1~holo4_003
visitdispersers_20230201_filt_BG40_65mm_1-holo4_003.list
OG550_65mm_1~holo4_003
visitdispersers_20230201_filt_OG550_65mm_1-holo4_003.list


/tmp/ipykernel_1877/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_1877/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_1877/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [ ]:
N = len(all_def_science_selected_final)

In [ ]:
if N>=1:
    n0=len(all_def_science_selected_final[0])
    display(all_def_science_selected_final[0])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
34263,2023020100387,AT_O_20230201_000387,20230201,387,2023-02-02 02:19:37.632008,2023-02-02 02:20:07.869,science,HD185975,empty~holo4_003,61.877403,30.0,306.283149,-87.489064,239.110801,spec,2.459978e+06,59977.096963
34292,2023020100388,AT_O_20230201_000388,20230201,388,2023-02-02 02:20:20.455997,2023-02-02 02:20:50.690,science,HD185975,empty~holo4_003,61.881655,30.0,306.283445,-87.489099,239.110333,spec,2.459978e+06,59977.097459
34296,2023020100396,AT_O_20230201_000396,20230201,396,2023-02-02 02:25:37.575983,2023-02-02 02:26:07.826,science,HD14943,empty~holo4_003,49.38575,30.0,35.719658,-51.132457,179.565079,spec,2.459978e+06,59977.101129
34268,2023020100397,AT_O_20230201_000397,20230201,397,2023-02-02 02:26:20.423990,2023-02-02 02:26:50.659,science,HD14943,empty~holo4_003,49.498615,30.0,35.719761,-51.132433,179.570724,spec,2.459978e+06,59977.101625
34272,2023020100405,AT_O_20230201_000405,20230201,405,2023-02-02 02:33:53.162995,2023-02-02 02:34:23.391,science,HD031128,empty~holo4_003,23.19883,30.0,73.02479,-27.102019,194.147909,spec,2.459978e+06,59977.106865
34301,2023020100406,AT_O_20230201_000406,20230201,406,2023-02-02 02:34:36.075013,2023-02-02 02:35:06.304,science,HD031128,empty~holo4_003,23.350399,30.0,73.024781,-27.101911,194.151589,spec,2.459978e+06,59977.107362
34305,2023020100414,AT_O_20230201_000414,20230201,414,2023-02-02 02:39:42.106988,2023-02-02 02:40:12.337,science,HD38949,empty~holo4_003,13.411111,30.0,87.056535,-24.49632,208.624201,spec,2.459978e+06,59977.110904
34308,2023020100415,AT_O_20230201_000415,20230201,415,2023-02-02 02:40:24.957007,2023-02-02 02:40:55.198,science,HD38949,empty~holo4_003,13.555257,30.0,87.056475,-24.496327,208.624039,spec,2.459978e+06,59977.111400
34312,2023020100423,AT_O_20230201_000423,20230201,423,2023-02-02 02:45:26.121014,2023-02-02 02:45:56.359,science,HD031128,empty~holo4_003,25.696552,30.0,73.024152,-27.101883,194.045058,spec,2.459978e+06,59977.114886
34341,2023020100424,AT_O_20230201_000424,20230201,424,2023-02-02 02:46:09.091009,2023-02-02 02:46:39.325,science,HD031128,empty~holo4_003,25.851661,30.0,73.02422,-27.10192,194.044853,spec,2.459978e+06,59977.115383


In [ ]:
if N>=2:
    n1=len(all_def_science_selected_final[1])
    display(all_def_science_selected_final[1])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
34293,2023020100389,AT_O_20230201_000389,20230201,389,2023-02-02 02:21:04.917997,2023-02-02 02:21:35.151,science,HD185975,BG40_65mm_1~holo4_003,61.88603,30.0,306.282736,-87.489105,239.11092,spec,2.459978e+06,59977.097974
34264,2023020100390,AT_O_20230201_000390,20230201,390,2023-02-02 02:21:47.855000,2023-02-02 02:22:18.091,science,HD185975,BG40_65mm_1~holo4_003,61.890273,30.0,306.282617,-87.48911,239.110915,spec,2.459978e+06,59977.098471
34297,2023020100398,AT_O_20230201_000398,20230201,398,2023-02-02 02:27:04.955995,2023-02-02 02:27:35.189,science,HD14943,BG40_65mm_1~holo4_003,49.614128,30.0,35.71983,-51.132398,179.564774,spec,2.459978e+06,59977.102141
34269,2023020100399,AT_O_20230201_000399,20230201,399,2023-02-02 02:27:47.978011,2023-02-02 02:28:18.216,science,HD14943,BG40_65mm_1~holo4_003,49.727128,30.0,35.719718,-51.132406,179.570731,spec,2.459978e+06,59977.102639
34273,2023020100407,AT_O_20230201_000407,20230201,407,2023-02-02 02:35:20.655017,2023-02-02 02:35:50.890,science,HD031128,BG40_65mm_1~holo4_003,23.512776,30.0,73.024772,-27.101898,194.15175,spec,2.459978e+06,59977.107878
34302,2023020100408,AT_O_20230201_000408,20230201,408,2023-02-02 02:36:03.468987,2023-02-02 02:36:33.702,science,HD031128,BG40_65mm_1~holo4_003,23.66791,30.0,73.024713,-27.102191,194.15104,spec,2.459978e+06,59977.108373
34306,2023020100416,AT_O_20230201_000416,20230201,416,2023-02-02 02:41:09.441980,2023-02-02 02:41:39.681,science,HD38949,BG40_65mm_1~holo4_003,13.706355,30.0,87.05655,-24.496321,208.62419,spec,2.459978e+06,59977.111915
34309,2023020100417,AT_O_20230201_000417,20230201,417,2023-02-02 02:41:52.262991,2023-02-02 02:42:22.496,science,HD38949,BG40_65mm_1~holo4_003,13.851102,30.0,87.056541,-24.496357,208.621953,spec,2.459978e+06,59977.112410
34313,2023020100425,AT_O_20230201_000425,20230201,425,2023-02-02 02:46:53.680990,2023-02-02 02:47:23.912,science,HD031128,BG40_65mm_1~holo4_003,26.014109,30.0,73.024162,-27.101897,194.044985,spec,2.459978e+06,59977.115899
34342,2023020100426,AT_O_20230201_000426,20230201,426,2023-02-02 02:47:36.593008,2023-02-02 02:48:06.850,science,HD031128,BG40_65mm_1~holo4_003,26.169292,30.0,73.024152,-27.101892,194.044912,spec,2.459978e+06,59977.116396


In [ ]:
if N>=3:
    n2=len(all_def_science_selected_final[2])
    display(all_def_science_selected_final[2])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
34265,2023020100391,AT_O_20230201_000391,20230201,391,2023-02-02 02:22:33.540006,2023-02-02 02:23:03.766,science,HD185975,OG550_65mm_1~holo4_003,61.894785,30.0,306.283088,-87.489104,239.110353,spec,2.459978e+06,59977.098999
34294,2023020100392,AT_O_20230201_000392,20230201,392,2023-02-02 02:23:16.342993,2023-02-02 02:23:46.575,science,HD185975,OG550_65mm_1~holo4_003,61.898981,30.0,306.282613,-87.489122,239.110666,spec,2.459978e+06,59977.099495
34298,2023020100400,AT_O_20230201_000400,20230201,400,2023-02-02 02:28:33.951007,2023-02-02 02:29:04.190,science,HD14943,OG550_65mm_1~holo4_003,49.847911,30.0,35.719757,-51.132398,179.570555,spec,2.459978e+06,59977.103171
34270,2023020100401,AT_O_20230201_000401,20230201,401,2023-02-02 02:29:16.869985,2023-02-02 02:29:47.101,science,HD14943,OG550_65mm_1~holo4_003,49.960861,30.0,35.719719,-51.132419,179.570616,spec,2.459978e+06,59977.103667
34274,2023020100409,AT_O_20230201_000409,20230201,409,2023-02-02 02:36:48.990003,2023-02-02 02:37:19.221,science,HD031128,OG550_65mm_1~holo4_003,23.830295,30.0,73.024762,-27.101978,194.148129,spec,2.459978e+06,59977.108900
34303,2023020100410,AT_O_20230201_000410,20230201,410,2023-02-02 02:37:31.802002,2023-02-02 02:38:02.041,science,HD031128,OG550_65mm_1~holo4_003,23.985419,30.0,73.024756,-27.102162,194.147665,spec,2.459978e+06,59977.109396
34338,2023020100418,AT_O_20230201_000418,20230201,418,2023-02-02 02:42:37.825005,2023-02-02 02:43:08.063,science,HD38949,OG550_65mm_1~holo4_003,14.002926,30.0,87.05655,-24.496324,208.624101,spec,2.459978e+06,59977.112938
34310,2023020100419,AT_O_20230201_000419,20230201,419,2023-02-02 02:43:20.652011,2023-02-02 02:43:50.891,science,HD38949,OG550_65mm_1~holo4_003,14.148308,30.0,87.05653,-24.496332,208.624084,spec,2.459978e+06,59977.113433
34314,2023020100427,AT_O_20230201_000427,20230201,427,2023-02-02 02:48:22.230982,2023-02-02 02:48:52.467,science,HD031128,OG550_65mm_1~holo4_003,26.335323,30.0,73.024112,-27.101895,194.045032,spec,2.459978e+06,59977.116924
34343,2023020100428,AT_O_20230201_000428,20230201,428,2023-02-02 02:49:05.048010,2023-02-02 02:49:35.285,science,HD031128,OG550_65mm_1~holo4_003,26.486866,30.0,73.024132,-27.101895,194.045093,spec,2.459978e+06,59977.117420


In [ ]:
if N>=4:
    n3=len(all_def_science_selected_final[3])
    display(all_def_science_selected_final[3])

In [ ]:
if N>=5:
    n4=len(all_def_science_selected_final[4])
    display(all_def_science_selected_final[4])

In [ ]:
df_forsave = pd.concat(all_def_science_selected_final)

In [ ]:
filename_out_csv = "holosummary_"+ str(date_sel) + ".csv"
filename_out_excel = "holosummary_"+ str(date_sel) + ".xlsx"

In [ ]:
filename_out_csv

'holosummary_20230201.csv'

In [ ]:
df_forsave.to_csv(filename_out_csv) 

In [ ]:
! pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
df_forsave.to_excel(filename_out_excel) 